In [8]:
!pip install python-telegram-bot --upgrade
!pip install asyncio


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.8/101.8 kB 7.9 MB/s eta 0:00:00


In [3]:
import nest_asyncio
import asyncio
from telegram.ext import ApplicationBuilder, CommandHandler, MessageHandler, filters
import tensorflow as tf
import cv2
import numpy as np


nest_asyncio.apply()


model = tf.keras.models.load_model("/content/model.h5")


class_mapping = {
    0: "Bluebell",
    1: "Buttercup",
    2: "Coltsfoot",
    3: "Cowslip",
    4: "Crocus",
    5: "Daffodil",
    6: "Daisy",
    7: "Dandelion",
    8: "Fritillary",
    9: "Iris",
    10: "Lilyvalley",
    11: "Pansy",
    12: "Snowdrop",
    13: "Sunflower",
    14: "Tigerlily",
    15: "Tulip",
    16: "Windflower"
}


TOKEN = ""


async def start(update, context):
    await update.message.reply_text("Welcome! Send me an image, and I will classify the flower.")


async def handle_image(update, context):

    photo = await update.message.photo[-1].get_file()
    photo_path = "received_image.jpg"
    await photo.download_to_drive(photo_path)


    image = cv2.imread(photo_path)
    image = cv2.resize(image, (224, 224))
    image = image / 255.0
    image = np.expand_dims(image, axis=0)


    predictions = model.predict(image)
    predicted_class = np.argmax(predictions)
    flower_name = class_mapping[predicted_class]


    await update.message.reply_text(f"The flower is: {flower_name}")


async def main():

    application = ApplicationBuilder().token(TOKEN).build()


    application.add_handler(CommandHandler("start", start))
    application.add_handler(MessageHandler(filters.PHOTO, handle_image))


    await application.run_polling()

if __name__ == "__main__":
    asyncio.run(main())


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 291ms/step


ERROR:telegram.ext.Application:No error handlers are registered, logging exception.
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/httpx/_transports/default.py", line 101, in map_httpcore_exceptions
    yield
  File "/usr/local/lib/python3.11/dist-packages/httpx/_transports/default.py", line 394, in handle_async_request
    resp = await self._pool.handle_async_request(req)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/httpcore/_async/connection_pool.py", line 256, in handle_async_request
    raise exc from None
  File "/usr/local/lib/python3.11/dist-packages/httpcore/_async/connection_pool.py", line 236, in handle_async_request
    response = await connection.handle_async_request(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/httpcore/_async/connection.py", line 103, in handle_async_request
    return await self._connection.handle_async_req

RuntimeError: Cannot close a running event loop